<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/algoritmos_gen%C3%A9ticos/optimizaci%C3%B3n_combinatoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from functools import partial
from collections import namedtuple
from random import choices, randint, randrange, random

In [2]:
def generate_genome(length):
    return choices([0, 1], k=length)

In [3]:
def generate_population(size, genome_length):
    return [generate_genome(genome_length) for _ in range(size)]

In [4]:
def single_point_crossover(a, b):
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

In [5]:
def mutation(genome, num=1, probability=0.5):
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

In [6]:
def population_fitness(population, fitness_func):
    return sum([fitness_func(genome) for genome in population])

In [7]:
def selection_pair(population, fitness_func):
    return choices(population=population, weights=[fitness_func(gene) for gene in population], k=2)

In [8]:
def sort_population(population, fitness_func):
    return sorted(population, key=fitness_func, reverse=True)

In [9]:
def genome_to_string(genome):
    return "".join(map(str, genome))

In [10]:
def print_stats(population, generation_id, fitness_func):
    print("Generación %02d" % generation_id)
    print("=============")
    print("Población: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Ajuste medio: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print("Mejor: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Peor: %s (%f)" % (genome_to_string(sorted_population[-1]), fitness_func(sorted_population[-1])))
    print("")
    return sorted_population[0]

In [11]:
def run_evolution(populate_func, fitness_func, fitness_limit, selection_func=selection_pair, crossover_func=single_point_crossover,
                  mutation_func=mutation, generation_limit=100, printer=print_stats):
    
    population = populate_func()

    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i

In [12]:
Thing = namedtuple('Thing', ['name', 'value', 'weight'])

In [13]:
def generate_things(num):
    return [Thing(f"thing{i}", i, i) for i in range(1, num+1)]

In [14]:
def fitness(genome, things, weight_limit):
    if len(genome) != len(things):
        raise ValueError("el genoma y los objetos deben ser de la misma longitud")

    weight = 0
    value = 0
    for i, thing in enumerate(things):
        if genome[i] == 1:
            weight += thing.weight
            value += thing.value

            if weight > weight_limit:
                return 0

    return value

In [15]:
first_example = [
    Thing('Laptop', 500, 2200),
    Thing('Audífonos', 150, 160),
    Thing('Taza de café', 60, 350),
    Thing('Cuaderno', 40, 333),
    Thing('Botella de agua', 30, 192),
]

second_example = first_example + [
    Thing('Dulces', 5, 25),
    Thing('Calcetines', 10, 38),
    Thing('Pañuelos', 15, 80),
    Thing('Celular', 500, 200),
    Thing('Batería', 100, 70)
]

In [16]:
things = generate_things(22)
things = second_example

weight_limit = 3000

print("Peso límite: %dkg" % weight_limit)

Peso límite: 3000kg


In [17]:
def bruteforce(things, weight_limit: int):
    if len(things) == 0:
        return 0, []

    max_value = 0
    max_valued_packed = []
    for i, thing in enumerate(things):
        if thing.weight > weight_limit:
            continue

        value, packed = bruteforce(things[i + 1:], weight_limit - thing.weight)
        if value + thing.value >= max_value:
            max_value = value + thing.value
            max_valued_packed = [thing] + packed

    return max_value, max_valued_packed

In [18]:
result = bruteforce(things, weight_limit)
result

(1310,
 [Thing(name='Laptop', value=500, weight=2200),
  Thing(name='Audífonos', value=150, weight=160),
  Thing(name='Botella de agua', value=30, weight=192),
  Thing(name='Dulces', value=5, weight=25),
  Thing(name='Calcetines', value=10, weight=38),
  Thing(name='Pañuelos', value=15, weight=80),
  Thing(name='Celular', value=500, weight=200),
  Thing(name='Batería', value=100, weight=70)])

In [19]:
population, generations = run_evolution(
		populate_func=partial(generate_population, size=10, genome_length=len(things)),
		fitness_func=partial(fitness, things=things, weight_limit=weight_limit),
		fitness_limit=result[0],
		generation_limit=100)

Generación 00
Población: [0110100011, 0100101110, 1001011101, 0001000111, 0001001010, 0011100101, 0011110000, 0000111100, 1110111101, 1011110010]
Ajuste medio: 386.000000
Mejor: 0110100011 (840.000000)
Peor: 1011110010 (0.000000)

Generación 01
Población: [1100101111, 0110100111, 0110100011, 0110100011, 0100101110, 0010100011, 0011001010, 0111100101, 0001100101, 1110100010]
Ajuste medio: 642.500000
Mejor: 1100101111 (1305.000000)
Peor: 1110100010 (0.000000)

Generación 02
Población: [1100101111, 1000101011, 0111100011, 0110100111, 0110100011, 0110100011, 0010101011, 0010100011, 0000100111, 0110100001]
Ajuste medio: 823.500000
Mejor: 1100101111 (1305.000000)
Peor: 0110100001 (340.000000)

Generación 03
Población: [1100101111, 1000101011, 1000100011, 0110101011, 0110100011, 0011100011, 0010100011, 0000001011, 0000100101, 1010101011]
Ajuste medio: 744.000000
Mejor: 1100101111 (1305.000000)
Peor: 1010101011 (0.000000)

Generación 04
Población: [1100101111, 1100101011, 1000101111, 100010101

In [20]:
def from_genome(genome, things):
    result = []
    for i, thing in enumerate(things):
        if genome[i] == 1:
            result += [thing]

    return result

In [21]:
def to_string(things):
    return f"[{', '.join([t.name for t in things])}]"


def value(things):
    return sum([t.value for t in things])


def weight(things):
    return sum([p.weight for p in things])

In [22]:
def print_stats2(things):
    print(f"Things: {to_string(things)}")
    print(f"Value {value(things)}")
    print(f"Weight: {weight(things)}")

In [23]:
sack = from_genome(population[0], things)
print_stats2(sack)

Things: [Laptop, Audífonos, Botella de agua, Dulces, Calcetines, Pañuelos, Celular, Batería]
Value 1310
Weight: 2965


In [24]:
result

(1310,
 [Thing(name='Laptop', value=500, weight=2200),
  Thing(name='Audífonos', value=150, weight=160),
  Thing(name='Botella de agua', value=30, weight=192),
  Thing(name='Dulces', value=5, weight=25),
  Thing(name='Calcetines', value=10, weight=38),
  Thing(name='Pañuelos', value=15, weight=80),
  Thing(name='Celular', value=500, weight=200),
  Thing(name='Batería', value=100, weight=70)])

**Por hacer**

* Añadir descripción de la solución
* Añadir planteamiento del problema
* Añadir comentarios a las funciones
* Reestructurar orden de funciones
* Añadir referencias